1. Estimate the channel capacity of plain old (wired) telephone service.

The Shannon capacity is $$ C = B\log_2(1+SNR) $$

The frequency spectrum of the human voice is from 300 to 3400 Hz and the typical SNR is 45dB according to https://www.jkaudio.com/article_03.htm 

In [2]:
import numpy as np

B = 3.1e3                       # Bandwidth in Hz   
SNR_dB = 45                     # signal to noise ratio in dB
SNR_linear = 10**(SNR_dB/10)    # linear conversion of SNR
C = B * np.log2(1 + SNR_linear) # Shannon capacity
print("The Shannon capacity is: ", C, "bits/s")

The Shannon capacity is:  46341.03834971382 bits/s
